In [1]:
import os, sys
home = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('__init__.py'))))
sys.path.append(home)

In [2]:
import ee
ee.Initialize()

In [3]:
import optical_datasources as optix
import harmonics

In [4]:
import pandas as pd
%matplotlib inline

In [5]:
states = ee.FeatureCollection("ft:17aT9Ud-YnGiXdXEJUyycH2ocUqreOeKGbzCkUw")

In [7]:
def export_lxharmonics(state_abbr, year, bands=None, rmbands=None, suffix=''):

    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year+1, 1, 1)

    fname = "lxharmonics_{0}_{1}{2}".format(state_abbr.lower(), str(year), suffix)
    region = ee.Feature(states.filterMetadata('id', 'equals', state_abbr).first())

    lxh = harmonics.lx_hregr(region.geometry(), start_date, end_date,
                             omega=1.5,
                             imgmask=imgmask,
                             bands=bands,
                             rmbands=rmbands,
                             addcount=True)

    lxh = lxh.set('year', year, 'state', state_abbr)

    expcoord = region.geometry().coordinates().getInfo()[0]  # this is still needed in python

    assetid = "projects/lobell-lab/harmonics_us/"+fname
    task = ee.batch.Export.image.toAsset(image=lxh.clip(region.geometry()),
                                         description=fname,
                                         assetId=assetid,
                                         region=expcoord,
                                         scale=30,
                                         crs='EPSG:3857',
                                         maxPixels=1e13)

    return task


## Export assets

In [10]:
for year in [2010, 2011, 2012, 2013, 2014, 2015, 2016]:
#     for st in ['NE', 'IL', 'IN', 'IA']:
    for st in ['ND', 'SD']:
        e = export_lxharmonics(st, year, 
                              bands=None, 
                              rmbands=['BLUE', 'RED', 'GREEN'], 
                              suffix='_novisbands')
        e.start()